In [1]:
import numpy as np
import cv2 as cv
import os

In [20]:
import gdown
import zipfile
import rarfile
import os

file_id = "1DCwmP8Y5ZEV_6P3Kd9Xf21-gsAXTHU4F"
file_url = f"https://drive.google.com/uc?id={file_id}"
output = "test.zip"

gdown.download(file_url, output, quiet=False)

rar_path = "test.zip"  # Replace with your RAR file name

with rarfile.RarFile(rar_path) as rar_ref:
    file_name = rar_ref.namelist()[0]  # Get the single file inside the RAR
    rar_ref.extract(file_name, ".")  # Extract to the current directory

    # If the extracted file has a subdirectory, move it to the current directory
    extracted_path = os.path.join(".", file_name)
    new_path = os.path.basename(file_name)
    os.rename(extracted_path, new_path)

Downloading...
From (original): https://drive.google.com/uc?id=1DCwmP8Y5ZEV_6P3Kd9Xf21-gsAXTHU4F
From (redirected): https://drive.google.com/uc?id=1DCwmP8Y5ZEV_6P3Kd9Xf21-gsAXTHU4F&confirm=t&uuid=a1ef2ac0-bb56-4b1b-bda4-b7bd36b49bba
To: C:\Users\Harsh\revision\computer_vision\finger\test.zip
100%|█████████████████████████████████████████████████████████████████████████████| 31.6M/31.6M [00:04<00:00, 7.82MB/s]


In [3]:
img_arr = []
feature = []

for img in os.listdir(__dirname):
    feature.append(int(img[-6:-5]))
    cur_img = cv.imread(os.path.join(__dirname, img))
    cur_img = cv.resize(cur_img, (224,224))
    cur_img = cv.cvtColor(cur_img, cv.COLOR_BGR2GRAY)
    cur_img = cv.threshold(cur_img, 110, 255, cv.THRESH_BINARY)[1]
    img_arr.append(cur_img)

In [4]:
img_arr = np.array(img_arr)
feature = np.array(feature)

In [5]:
img_arr.shape, feature.shape

((3600, 224, 224), (3600,))

In [6]:
import tensorflow as tf
from tensorflow import keras

In [7]:
model = keras.models.load_model("finger.keras")

In [8]:
model.summary()


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d_3 (Conv2D)                    │ (None, 222, 222, 128)       │           1,280 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_3 (MaxPooling2D)       │ (None, 111, 111, 128)       │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_4 (Conv2D)                    │ (None, 109, 109, 64)        │          73,792 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_4 (MaxPooling2D)       │ (None, 54, 54, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_5 (Conv2D)                    │ (None, 52, 52, 32)          │          18,464 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_5 (MaxPooling2D)       │ (None, 26, 26, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 26, 26, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_1 (Flatten)                  │ (None, 21632)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_4 (Dense)                      │ (None, 128)                 │       2,769,024 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_5 (Dense)                      │ (None, 64)                  │           8,256 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_6 (Dense)                      │ (None, 32)                  │           2,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_7 (Dense)                      │ (None, 6)                   │             198 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 8,619,284 (32.88 MB)

 Trainable params: 2,873,094 (10.96 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 5,746,190 (21.92 MB)

In [9]:
img_arr_pred = np.expand_dims(img_arr, axis=-1)

In [10]:
img_arr_pred.shape

(3600, 224, 224, 1)

In [11]:
y_pred = model.predict(img_arr_pred)

113/113 ━━━━━━━━━━━━━━━━━━━━ 52s 455ms/step


In [12]:
y_pred = np.argmax(y_pred, axis=1)

In [13]:
print(f"accuracy = {np.mean(y_pred == feature)}")

accuracy = 0.8583333333333333


In [14]:
def random_image():
    term = int(np.random.rand()*len(img_arr_pred))
    print(feature[term])
    return img_arr[term]

In [15]:
# self images

In [16]:

def get_img(string):
    self_img = rf"C:\Users\Harsh\Downloads\{string}.jpeg"
    self_img = cv.imread(self_img)

    if self_img is None:
        print(f"Error: Could not load {self_img}")
        return None

    # Crop first (ensure cropping region is valid)
    self_img = self_img[300:800, 50:500]  # Crops (500, 450)

    # Resize to (224, 224)
    self_img = cv.resize(self_img, (224, 224))
    self_img = cv.cvtColor(self_img, cv.COLOR_BGR2GRAY)
    self_img = cv.threshold(self_img, 145, 255, cv.THRESH_BINARY_INV)[1]
    
    return self_img

In [17]:
names = ["4","5'","3'"]

In [18]:
for name in names:
    cur_img = get_img(name)
    cv.imshow("cur", cur_img)
    cv.waitKey(0)
    cur_img = np.expand_dims(cur_img, axis=-1)
    cur_img = np.expand_dims(cur_img, axis=0)
    print(np.argmax(model.predict(cur_img), axis=1))
cv.destroyAllWindows()


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
[4]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
[5]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
[5]


In [19]:
some = random_image()
cv.imshow("some",some)
cv.waitKey(0)

4


-1